In [1]:
from google.colab import drive
import os
import pandas as pd

In [3]:
current_directory = "/data"
os.chdir(current_directory)

In [4]:
!pip install ntlk
!pip install num2words
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install ruptures
!pip install sentence_transformers
!pip install fastcoref

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement ntlk (from versions: none)
ERROR: No matching distribution found for ntlk
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem import PorterStemmer
from nltk import word_tokenize, sent_tokenize, pos_tag, ne_chunk
from nltk import RegexpParser
from nltk import Tree

import re
import num2words
import json

#For segmentation of transcript into product reviews
import ruptures as rpt
from ruptures.base import BaseCost
from sentence_transformers import SentenceTransformer, util
from fastcoref import spacy_component
import spacy
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("fastcoref")

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [6]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

Intialise Classes for Product Review Identification and Segmentation

In [7]:
class ProductIdentifier:
  def __init__(self):
    pass

  def remove_non_ascii(self, text):
    return re.sub(r'[^\x00-\x7F]+',' ', text)

  def GetNounsWithNLTK(self, text, chunk_func=ne_chunk):
    chunked = chunk_func(pos_tag(word_tokenize(text)))
    continuous_chunk = []
    current_chunk = []

    for subtree in chunked:
      if type(subtree) == Tree:
        current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
      elif current_chunk:
        named_entity = " ".join(current_chunk)
        if named_entity not in continuous_chunk:
          continuous_chunk.append(named_entity)
          current_chunk = []
      else:
        continue

    return continuous_chunk

  def Check_If_NP_Is_Known_Product(self, np, product_lexicon):
    for product in product_lexicon:
      if np == product:
        return True
    return False

  def Check_If_NP_Has_Product_Indicator(self, np, product_indicators):
    for product in product_indicators:
      similarity = fuzz.partial_ratio(np.lower(),product.lower())
      if similarity >= 80:
        return True
    return False

  def Check_If_NP_Is_Product(self, np, product_indicators, product_lexicon):
    if self.Check_If_NP_Is_Known_Product(np, product_lexicon):
      return True
    elif self.Check_If_NP_Has_Product_Indicator(np, product_indicators):
      return True
    else:
      return False

  def is_not_substring_of_another_product(self, product, product_lst):
    for comparison_product in product_lst:
      if product.lower() == comparison_product.lower():
        continue
      if product.lower() in comparison_product.lower():
        return False
    return True

  def detect_products_from_transcript(self, review_dict, product_indicators, product_lexicon):
    text = review_dict["transcript_text"]
    #likely to contain the products
    nouns_in_transcript = self.GetNounsWithNLTK(text)

    #only keep if len > 2 
    nouns_in_transcript = list(filter(lambda noun_phrase: len(nltk.word_tokenize(noun_phrase))>=2, nouns_in_transcript))

    #keep if match with brand name or contain a product term
    nouns_in_transcript = list(filter(lambda noun_phrase: self.Check_If_NP_Is_Product(noun_phrase, product_indicators, product_lexicon), nouns_in_transcript))

    #filter the products
    nouns_in_transcript = list(filter(lambda product: self.is_not_substring_of_another_product(product, nouns_in_transcript), nouns_in_transcript))

    return list(set(nouns_in_transcript))
  
  def contains_url(self, text):
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex, text)
    if url != []:
      return True
    else:
      return False

  def remove_url(self, text):
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    removed_url_text = re.sub(regex, "", text)
    return removed_url_text

  def clean_desc(self, text):
    text = text.replace(":", "")
    text = text.replace("*", "")
    return text

  def detect_products_from_desc(self, review_dict):
    description = review_dict["description"]
    channel_title = review_dict["channel_title"]
    split_desc = description.split("\n")
    desc_sentence_with_url = list(filter(lambda txt: self.contains_url(txt), split_desc))
    #set higher threshold because even if e.g. "mal" "ibu" match seperately -- it raises the match score
    desc_sentence_without_channel_url = list(filter(lambda txt: fuzz.partial_ratio(txt.lower(),channel_title.lower()) < 80, desc_sentence_with_url))
    products = list(filter(lambda txt: self.Check_If_NP_Is_Product(txt, product_indicators, product_lexicon),desc_sentence_without_channel_url))
    products = list(filter(lambda txt: self.is_not_substring_of_another_product(txt,products),products))
    filtered_desc_product_lst = [self.clean_desc(self.remove_url(txt)) for txt in products]
    return filtered_desc_product_lst

  def detect_products(self, review_dict, product_indicators, product_lexicon):
    filtered_transcript_product_lst = self.detect_products_from_transcript(self, review_dict, product_indicators, product_lexicon)
    filtered_desc_product_lst = self.detect_products_from_desc(review_dict)
    filtered_transcript_product_lst.extend(filtered_desc_product_lst)
    filtered_product_lst = list(filter(lambda txt: self.is_not_substring_of_another_product(txt,filtered_transcript_product_lst),filtered_transcript_product_lst))
    filtered_product_lst = list(set(filtered_product_lst))
    return filtered_product_lst

class CoreferenceResolver:
  def __init__(self):
    pass
  
  def get_coresolved_text(self,texts):
    docs = list(nlp.pipe(texts, component_cfg={"fastcoref": {'resolve_text': True}}))
    coresolved_transcript = docs[0]._.resolved_text
    return coresolved_transcript 

#Define cost function for segmentation
class CosineCost(BaseCost):
    """Cost derived from the cosine similarity."""

    # The 2 following attributes must be specified for compatibility.
    model = "custom_cosine"
    min_size = 2

    def fit(self, signal):
        """Set the internal parameter."""
        self.signal = signal
        self.gram = util.cos_sim(signal, signal)
        return self

    def error(self, start, end) -> float:
        """Return the approximation cost on the segment [start:end].

        Args:
            start (int): start of the segment
            end (int): end of the segment
        Returns:
            segment cost
        Raises:
            NotEnoughPoints: when the segment is too short (less than `min_size` samples).
        """
        if end - start < self.min_size:
            raise NotEnoughPoints
        sub_gram = self.gram[start:end, start:end]
        val = sub_gram.diagonal().sum()
        val -= sub_gram.sum() / (end - start)
        return val

class TranscriptSegmenter:
  def __init__(self):
    self.segmentation_cost_function_class = CosineCost()
    self.product_identifier = ProductIdentifier()
    self.sentence_embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

  def get_sentence_list_from_transcript(self, transcript):
    phrases = nltk.sent_tokenize(transcript)
    pos_tagged = [pos_tag(nltk.word_tokenize(sentence.lower())) for sentence in phrases]
    new_combined_sentence_list = []
    for i in range(len(phrases)):
      tagged_phrase = pos_tagged[i]
      tag_of_first_word = tagged_phrase[0][1]
      conjunctions_lst = ["IN", "CC", "WDT", "TO", "WRB", "VBG", "VBZ", "RB"]
      end_lst = ["IN", "CC", "WDT", "TO", "WRB", "VBG", "VBZ", "DT", "RB"]
      join_sentence = False
      if i >= 1:
        last_tag_in_prev_sentence = pos_tagged[i-1][-1][1]
        if last_tag_in_prev_sentence in end_lst:
          join_sentence = True
      if tag_of_first_word in conjunctions_lst:
        join_sentence = True

      if join_sentence:
        #lowercase the first letter to reduce noise when doing NP identification e.g. "Which is so beautiful" --> Which to which
        sub_clause = phrases[i].replace(phrases[i][0],phrases[i][0].lower(),1)
        new_combined_sentence_list[-1] = new_combined_sentence_list[-1] + " " + sub_clause
      else:
        new_combined_sentence_list.append(phrases[i])
    return new_combined_sentence_list

  def segment_transcript(self, num_products, resolved_sentences, original_sentences):
    #Find breaks where similarity between chunks is the lowest
    embeddings = self.sentence_embedding_model.encode(resolved_sentences, convert_to_tensor=True)
    n_bkps = num_products-1+2  # e.g. there are 9 change points (10 text segments) + intro and outro
    algo = rpt.Dynp(custom_cost=self.segmentation_cost_function_class, min_size=2, jump=1).fit(embeddings)
    predicted_bkps = algo.predict(n_bkps=n_bkps)
    iteration_lst = predicted_bkps
    iteration_lst.insert(0, 0)

    chunk_lst = []
    for i in range(len(iteration_lst)-1):
      chunk_resolved = " ".join(resolved_sentences[iteration_lst[i]:iteration_lst[i+1]])
      chunk_org = " ".join(original_sentences[iteration_lst[i]:iteration_lst[i+1]])
      chunk_lst.append((chunk_resolved, chunk_org))
    return chunk_lst

  def split_transcript_by_detected_products(self, review_dict, product_indicators, product_lexicon):
    text_transcript = review_dict["transcript_text"]

    #Conduct coresolution to better identify products in each segmented chunk and improve distinction between different chunks
    coresolved_transcript = self.coreference_resolver.get_coreference_resolved_text(text_transcript)

    #Get an estimate of the number of products for segmentation algo
    filtered_product_lst = self.product_identifier.detect_products(coresolved_transcript, product_indicators, product_lexicon)
    #to remove as channel title and other human names that may be identified as a NP
    channel_title = review_dict["channel_title"]
    stop_words = channel_title.split()
    filtered_product_lst = list(filter(lambda noun_phrase: noun_phrase not in stop_words, filtered_product_lst))

    #Conduct segmentation
    sentences = self.get_sentences_from_transcript(coresolved_transcript)
    num_products= len(filtered_product_lst)
    chunk_lst = self.segment_transcript(num_products, sentences)

    product_review_dict = {}
    for chunk_pair in chunk_lst[1:len(chunk_lst)-1]:
      for product in filtered_product_lst:
        resolved_chunk = chunk_pair[0]
        original_chunk = chunk_pair[1]
        if fuzz.partial_ratio(product, resolved_chunk) > 70:
          if product in product_review_dict:
            product_review_dict[product] = product_review_dict[product] + " " + original_chunk
          else: 
            product_review_dict[product] = original_chunk

    return product_review_dict

  #Case of video having chapters

  def process_raw_transcript(self, raw_transcript):
    text_list = [segment["text"] for segment in raw_transcript]
    text = " ".join(text_list)
    text = re.sub("\n", " ", text)
    #remove non-ascii characters -- to remove foreign lang characters
    text = self.product_identifier.remove_non_ascii(text)
    return text

  def split_transcript_by_chapters(self, raw_transcript, chapters):
    starts = [round(segment["start"]) for segment in raw_transcript]
    product_review_dict = []
    chapter_index_list = []
    for chapter in chapters:
      time_in_sec = round(chapter["time_in_min"] * 60)
      #get a index list of chapters 
      new_insertion_index = bisect.bisect_left(starts, time_in_sec)
      chapter_index_list.append(new_insertion_index)
    chapter_index_list.append(len(raw_transcript)-1)
    for i in range(len(chapter_index_list)-1):
      title = chapters[i]["title"]
      start_index = chapter_index_list[i]
      end_index = chapter_index_list[i+1]
      product_review_dict.append({title:self.process_raw_transcript(raw_transcript[start_index:end_index])})
    
    return product_review_dict


  def get_product_review_dict(self, review_dict, product_indicator, product_lexicon):
    chapters = sample_review_dict["chapters"]
    product_review_dict = {}
    if chapters == []:
      product_review_dict = self.split_transcript_by_detected_products(review_dict, product_indicator, product_lexicon)
    else:
      raw_transcript = sample_review_dict["transcript_raw"]
      product_review_dict = self.split_transcript_by_chapters(raw_transcript, chapters)
    return product_review_dict


Load Youtube Data

In [8]:
import bisect
import json
from fuzzywuzzy import fuzz

In [9]:
product_identifier = ProductIdentifier()
transcript_segmenter = TranscriptSegmenter()

In [10]:
#Load data scraped using various Youtube API
with open('youtube_data.json', 'r') as f:
  reviews = json.load(f)

How it works
- Chapter segmentation remains the same

In [11]:
sample_review_id = "8qvjvbYvYsA"
sample_review_dict = reviews[sample_review_id]

In [12]:
product_indicators = pd.read_csv("Category Lexicon.csv")["Category"].to_list()
product_lexicon = pd.read_csv("Product Lexicon.csv")["Product"].to_list()

Addition from Iteration 1: Additional parsing of sentences from NLTK because transcript is in sub-sentences e.g. "This is", "A primer", "Which is wonderful" so we want to join these sub-sentences together to improve segmentation

In [13]:
#captured in transcript_segmenter.get_sentence_list_from_transcript(transcript)
text_transcript = sample_review_dict["transcript_text"]
phrases = nltk.sent_tokenize(text_transcript)
pos_tagged = [pos_tag(nltk.word_tokenize(sentence.lower())) for sentence in phrases]
new_combined_sentence_list = []
for i in range(len(phrases)):
  tagged_phrase = pos_tagged[i]
  tag_of_first_word = tagged_phrase[0][1]
  conjunctions_lst = ["IN", "CC", "WDT", "TO", "WRB", "VBG", "VBZ", "RB"]
  end_lst = ["IN", "CC", "WDT", "TO", "WRB", "VBG", "VBZ", "DT", "RB"]
  join_sentence = False
  if i >= 1:
    last_tag_in_prev_sentence = pos_tagged[i-1][-1][1]
    if last_tag_in_prev_sentence in end_lst:
      join_sentence = True
  if tag_of_first_word in conjunctions_lst:
    join_sentence = True

  if join_sentence:
    #lowercase the first letter to reduce noise when doing NP identification e.g. "Which is so beautiful" --> Which to which
    sub_clause = phrases[i].replace(phrases[i][0],phrases[i][0].lower(),1)
    new_combined_sentence_list[-1] = new_combined_sentence_list[-1] + " " + sub_clause
  else:
    new_combined_sentence_list.append(phrases[i])
print(f"Before combining: {len(phrases)}")
print(f"After combining: {len(new_combined_sentence_list)}")

Before combining: 181
After combining: 139


Addition from Iteration 1: Product Identification from Description
- Keep sentences with URL in the description
- We remove sentences with channel url
- We then parse for noun phrases and keep those that are related to makeup as 
- These are likely the product names

In [14]:
product_identifier = ProductIdentifier()
filtered_desc_product_lst = product_identifier.detect_products_from_desc(sample_review_dict)
filtered_desc_product_lst

['Primer elf Oil Control Lotion ',
 'Foundation L’Oreal Skin Serum (0.5-2 Very Light) ',
 'Concealer NYX Serum Concealer (Vanilla) ',
 'Setting Powder Maybelline Fit Me Loose Finishing Powder (Fair Light) ',
 'Eyebrow Winky Lux Micro Universal Brow Pencil ',
 'Eyebrow Gel NYX The Brow Glue ',
 'Eyeshadow Primer Milani Eyeshadow Primer ',
 'Eyeshadow Primer Wet n Wild PhotoFocus Base  ',
 'Eyeshadow elf Bite Size Palette (Truffles, Rose Water) ',
 'Gel Eyeliner Milani Stay Put Eyeliner ',
 'Gel Eyeliner Revlon Vinyl Eyeliner ',
 'Mascara Primer L’Oreal Voluminous Primer ',
 'Mascara Maybelline Lash Sensational ',
 'Cream Bronzer elf Putty Bronzer (Tan Lines) ',
 'Bronzer Juvia’s Place Bronzer Palette (Light) ',
 'Cream Blush Elf Putty Blush (Tahiti) ',
 'Blush Milani Powder Blush (Blossomtime Rose) ',
 'Highlighter Catrice More Than Glow Highlighter (Supreme Rose Beam) ',
 'Lip Liner NYX Lip Pencil (Nude Truffle) ',
 'Lip Liner Milani Color Statement Lip Liner (04 All Natural) ',
 'Lip 

Same as Iteration 1: Parse for product names in text transcript

In [15]:
#We then also parse for noun phrases in the text transcript using syntax rules -- this is same as iteration 1
filtered_transcript_product_lst = product_identifier.detect_products_from_transcript(sample_review_dict, product_indicators, product_lexicon)
filtered_transcript_product_lst

['Me Concealer Serum',
 'Blossomtime Rose',
 'Tan Lines',
 'Milani Blush',
 'NYX Bear',
 'Maybelline Age Rewind Concealer',
 'Happy New Year',
 'Maybelline Fit Me',
 'Putty Blush',
 'Putty Bronzer',
 'Match Hyaluronic Tinted Serum']

Combine both the products in the description and the transcript

In [16]:
filtered_transcript_product_lst.extend(filtered_desc_product_lst)
filtered_product_lst = list(filter(lambda txt: product_identifier.is_not_substring_of_another_product(txt,filtered_transcript_product_lst),filtered_transcript_product_lst))
filtered_product_lst = list(set(filtered_product_lst))
filtered_product_lst

['Bronzer Juvia’s Place Bronzer Palette (Light) ',
 'Primer elf Oil Control Lotion ',
 'Mascara Maybelline Lash Sensational ',
 'Happy New Year',
 'Mascara Primer L’Oreal Voluminous Primer ',
 'Milani Blush',
 'NYX Bear',
 'Lip Liner NYX Lip Pencil (Nude Truffle) ',
 'Maybelline Age Rewind Concealer',
 'Eyeshadow elf Bite Size Palette (Truffles, Rose Water) ',
 'Lip Gloss Milani Keep it Full Maxxx Lip Plumper (110 Little Secret) ',
 'Foundation L’Oreal Skin Serum (0.5-2 Very Light) ',
 'Match Hyaluronic Tinted Serum',
 'Highlighter Catrice More Than Glow Highlighter (Supreme Rose Beam) ',
 'Gel Eyeliner Revlon Vinyl Eyeliner ',
 'Blush Milani Powder Blush (Blossomtime Rose) ',
 'Lip Liner Milani Color Statement Lip Liner (04 All Natural) ',
 'Setting Powder Maybelline Fit Me Loose Finishing Powder (Fair Light) ',
 'Gel Eyeliner Milani Stay Put Eyeliner ',
 'Cream Bronzer elf Putty Bronzer (Tan Lines) ',
 'Cream Blush Elf Putty Blush (Tahiti) ',
 'Me Concealer Serum',
 'Eyebrow Gel NYX 

Experiment with a faster model for model based coreference resolution 

In [17]:
texts = [text_transcript]
docs = list(nlp.pipe(texts, component_cfg={"fastcoref": {'resolve_text': True}}))
coresolved_transcript = docs[0]._.resolved_text
coresolved_transcript

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

'- Hello, my lovelies, and Happy New Year.\nToday, it\'s all about the\nbest affordable makeup of 2021\nfrom my rendition, my opinion, my makeup\nthat I just kept going\ntowards, kept reaching towards,\nand I just couldn\'t get enough of.\nSome of them are newer products,\neither the beginning of the year, mid,\nor even towards the end of the year,\nand then some of them are\njust rediscovered loves.\nLet\'s begin, starting\nfirst with the primer.\nThis right here is by e.l.f.\nI don\'t usually like primers,\nbut this one I actually\n(laughs) really, really enjoyed.\nthis one is their Oil Control\nLiquid Lotion with tea tree oil.\nthis one might put you off\nif you have dry skin,\nbut I found that this one just helped balance\nthe oil throughout the\nday, especially in a humid,\nwarm, hot climate,\ndefinitely towards the\nspring and summertime,\nbut I really enjoyed this\nthroughout the entire year,\nand I would mostly focus in areas\nwhere I would get extra shiny,\nlike right here, th

Get the sentence list from the coresolved transcript as we want to do chunking based on the resolved transcript since the sentences from different product reviews are likely to be more different
- We will keep the original sentence list because we want to retain both the original and the resolved chunks

In [18]:
resolved_list = transcript_segmenter.get_sentence_list_from_transcript(coresolved_transcript)
resolved_list

['- Hello, my lovelies, and Happy New Year.',
 "Today, it's all about the\nbest affordable makeup of 2021\nfrom my rendition, my opinion, my makeup\nthat I just kept going\ntowards, kept reaching towards,\nand I just couldn't get enough of.",
 'Some of them are newer products,\neither the beginning of the year, mid,\nor even towards the end of the year,\nand then some of them are\njust rediscovered loves.',
 "Let's begin, starting\nfirst with the primer.",
 'This right here is by e.l.f.',
 "I don't usually like primers,\nbut this one I actually\n(laughs) really, really enjoyed.",
 'this one is their Oil Control\nLiquid Lotion with tea tree oil.',
 "this one might put you off\nif you have dry skin,\nbut I found that this one just helped balance\nthe oil throughout the\nday, especially in a humid,\nwarm, hot climate,\ndefinitely towards the\nspring and summertime,\nbut I really enjoyed this\nthroughout the entire year,\nand I would mostly focus in areas\nwhere I would get extra shiny,\nl

Same as Iteration 1: Segmentation by finding breakpoints between sentences with least semantic similarity between the sentence embeddings

In [19]:
num_products= len(filtered_product_lst)
chunk_lst = transcript_segmenter.segment_transcript(num_products, resolved_list, new_combined_sentence_list)

In [20]:
for i in range(len(chunk_lst)):
  chunk_pair = chunk_lst[i]
  print(f"Chunk {i}")
  print(chunk_pair[1].replace("\n", " "))

Chunk 0
- Hello, my lovelies, and Happy New Year. Today, it's all about the best affordable makeup of 2021 from my rendition, my opinion, my makeup that I just kept going towards, kept reaching towards, and I just couldn't get enough of. Some of them are newer products, either the beginning of the year, mid, or even towards the end of the year, and then some of them are just rediscovered loves.
Chunk 1
Let's begin, starting first with the primer. This right here is by e.l.f. I don't usually like primers, but this one I actually (laughs) really, really enjoyed.
Chunk 2
This is their Oil Control Liquid Lotion with tea tree oil. This might put you off if you have dry skin, but I found that this just helped balance the oil throughout the day, especially in a humid, warm, hot climate, definitely towards the spring and summertime, but I really enjoyed this throughout the entire year, and I would mostly focus in areas where I would get extra shiny, like right here, the T-zone, or just even us

Change from Iteration 1: Map from Product to Chunk if Chunk contains product name rather than using semantic matching with sentence transformers

In [21]:
product_review_dict = {}
for chunk_pair in chunk_lst:
  for product in filtered_product_lst:
    resolved_chunk = chunk_pair[0]
    original_chunk = chunk_pair[1]
    if fuzz.partial_ratio(product, resolved_chunk) > 70:
      if product in product_review_dict:
        product_review_dict[product] = product_review_dict[product] + " " + original_chunk
      else: 
        product_review_dict[product] = original_chunk

In [22]:
for k, v in product_review_dict.items():
  print(k)
  print(v.replace("\n", " "))

Happy New Year
- Hello, my lovelies, and Happy New Year. Today, it's all about the best affordable makeup of 2021 from my rendition, my opinion, my makeup that I just kept going towards, kept reaching towards, and I just couldn't get enough of. Some of them are newer products, either the beginning of the year, mid, or even towards the end of the year, and then some of them are just rediscovered loves.
Primer elf Oil Control Lotion 
This is their Oil Control Liquid Lotion with tea tree oil. This might put you off if you have dry skin, but I found that this just helped balance the oil throughout the day, especially in a humid, warm, hot climate, definitely towards the spring and summertime, but I really enjoyed this throughout the entire year, and I would mostly focus in areas where I would get extra shiny, like right here, the T-zone, or just even use it all over my face if I knew it was gonna be really hot and humid that day. and it also smells really nice. but it doesn't really mattif